In [23]:
import numpy as np
import phate
import sys
sys.path.append('../../src/')
from data_script import hemisphere_data, sklearn_swiss_roll
from data_convert import convert_data
from plotly3d.plot import scatter
from negative_sampling import add_negative_samples, make_hi_freq_noise
import pathlib
from scipy.spatial.distance import pdist, squareform
from sklearn.model_selection import train_test_split
import phate
import numpy as np
from scipy.spatial.distance import pdist, squareform
from omegaconf import DictConfig, OmegaConf
import hydra

Generate data

In [24]:
gt_X, X, _ = sklearn_swiss_roll(n_samples=1000, noise=0.0)

In [25]:
def convert_data(X, seed=42, test_size=0.2):
    phate_op = phate.PHATE(random_state=42, knn=20, t=30, n_components=3, decay=5)
    phate_data = phate_op.fit_transform(X)
    colors = np.zeros(X.shape[0])
    dists_all = squareform(pdist(phate_op.diff_potential))
    ids = np.arange(0, X.shape[0])
    id_train, id_test = train_test_split(ids, test_size=test_size, random_state=seed)
    is_train = np.isin(ids, id_train)
    X_train = X[is_train]
    phate_op = phate.PHATE(random_state=42, knn=20, t=30, n_components=3, decay=5)
    _ = phate_op.fit_transform(X_train)
    dists = dists_all.copy()
    dists_train = squareform(pdist(phate_op.diff_potential))
    dists[is_train][:,is_train] = dists_train
    return dict(
        data=X,
        colors=colors,
        dist=dists,
        dist_all=dists_all, # deprecated.
        phate=phate_data,
        is_train=is_train
    )

In [26]:
data_dict0 = convert_data(X)

Calculating PHATE...
  Running PHATE on 1000 observations and 3 variables.
  Calculating graph and diffusion operator...
    Calculating KNN search...
    Calculated KNN search in 0.04 seconds.
    Calculating affinities...
    Calculated affinities in 0.02 seconds.
  Calculated graph and diffusion operator in 0.07 seconds.
  Calculating diffusion potential...


  Calculated diffusion potential in 0.73 seconds.
  Calculating metric MDS...


/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/sklearn/manifold/_mds.py:299: FutureWarning:

The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.



  Calculated metric MDS in 17.83 seconds.
Calculated PHATE in 18.65 seconds.
Calculating PHATE...
  Running PHATE on 800 observations and 3 variables.
  Calculating graph and diffusion operator...
    Calculating KNN search...
    Calculated KNN search in 0.04 seconds.
    Calculating affinities...
    Calculated affinities in 0.02 seconds.
  Calculated graph and diffusion operator in 0.07 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 0.26 seconds.
  Calculating metric MDS...


/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/sklearn/manifold/_mds.py:299: FutureWarning:

The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.



  Calculated metric MDS in 2.37 seconds.
Calculated PHATE in 2.71 seconds.


In [27]:
scatter(data_dict0['phate'], s=5)

In [28]:
scatter(data_dict0['data'], s=5)

In [29]:
data_name = 'swiss_roll_sklearn'
np.savez(f"../../data/{data_name}.npz", **data_dict0)

In [30]:
seed = 32
data_name = 'swiss_roll_sklearn'
noise_types = ['hi-freq-no-add']
noise_type = 'hi-freq-no-add'
noise_level = 1.
mask_dist = False
data_dict = add_negative_samples(data_dict0.copy(), subset_rate=1., noise_rate=noise_level, seed=seed, noise=noise_type, mask_dists=mask_dist, shell=True)
save_folder = f'../../data/negative_sampling_toy_shell/{mask_dist}/'
pathlib.Path(save_folder).mkdir(parents=True, exist_ok=True)
np.savez(f"{save_folder}/{data_name}.npz", **data_dict)
scatter(data_dict['data'], data_dict['mask_x'], s=2)